<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2021notebooks/2021_0618CAM_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# source: pytorchCAM.py
# simple implementation of CAM in PyTorch for the networks such as ResNet, DenseNet, SqueezeNet, Inception

import io
import requests
from PIL import Image
from torchvision import models, transforms
from torch.autograd import Variable
from torch.nn import functional as F
import numpy as np
import cv2
import pdb

# input image
LABELS_URL = 'https://s3.amazonaws.com/outcome-blog/imagenet/labels.json'
IMG_URL = 'http://media.mlive.com/news_impact/photo/9933031-large.jpg'

# networks such as googlenet, resnet, densenet already use global average pooling at the end, so CAM could be used directly.
model_id = 2
if model_id == 1:
    net = models.squeezenet1_1(pretrained=True)
    finalconv_name = 'features' # this is the last conv layer of the network
elif model_id == 2:
    net = models.resnet18(pretrained=True)
    finalconv_name = 'layer4'
elif model_id == 3:
    net = models.densenet161(pretrained=True)
    finalconv_name = 'features'

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
net.eval()

In [3]:
# hook the feature extractor
features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

net._modules.get(finalconv_name).register_forward_hook(hook_feature)

# get the softmax weight
params = list(net.parameters())
weight_softmax = np.squeeze(params[-2].data.numpy())

def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

In [4]:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Resize((224,224)),
   transforms.ToTensor(),
   normalize
])

In [5]:
response = requests.get(IMG_URL)
img_pil = Image.open(io.BytesIO(response.content))
img_pil.save('test.jpg')

In [ ]:
import IPython
IPython.display.Image(url=IMG_URL)

In [7]:
img_tensor = preprocess(img_pil)
img_variable = Variable(img_tensor.unsqueeze(0))
logit = net(img_variable)

In [9]:
LABELS_ja_URL= 'https://gist.githubusercontent.com/PonDad/4dcb4b242b9358e524b4ddecbee385e9/raw/dda9454f74aa4fafee991ca8b848c9ab6ae0e732/imagenet_class_index.json'
classes_ja = {int(i):val['ja'] for i, val in enumerate(requests.get(LABELS_ja_URL).json())}

In [10]:
h_x = F.softmax(logit, dim=1).data.squeeze()
probs, idx = h_x.sort(0, True)
probs = probs.numpy()
idx = idx.numpy()

# output the prediction
for i in range(0, 5):
    #print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))
    print('{:.3f} -> {}'.format(probs[i], classes_ja[idx[i]]))

0.437 -> マウンテンバイク
0.131 -> アルプス
0.119 -> 自転車
0.108 -> 一輪車
0.049 -> 湖畔


In [11]:
classes = classes_ja

In [ ]:
# generate class activation mapping for the top1 prediction
candidate = 0
CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[candidate]])

# render the CAM and output
print('CAM の出力 {0}: {1}'. format(candidate,  classes[idx[candidate]]))

img = cv2.imread('test.jpg')
height, width, _ = img.shape
heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)
result = heatmap * 0.3 + img * 0.5
cv2.imwrite('CAM.jpg', result)
IPython.display.Image('CAM.jpg')

In [13]:
import matplotlib.pyplot as plt
from PIL import Image as pil_img
%matplotlib inline

In [ ]:
for candidate in range(5):
    # generate class activation mapping for the top1 prediction
    CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[candidate]])

    # render the CAM and output
    print('CAM の出力 {0}: {1}'. format(candidate,  classes[idx[candidate]]))

    img = cv2.imread('test.jpg')
    height, width, _ = img.shape
    heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)
    result = heatmap * 0.3 + img * 0.5
    cv2.imwrite('CAM.jpg', result)
    #IPython.display.Image('CAM.jpg')
    plt.axis(False); plt.imshow(pil_img.open('CAM.jpg'))
    plt.show()

In [ ]:
img_tensor = preprocess(img_pil)
img_variable = Variable(img_tensor.unsqueeze(0))
logit = net(img_variable)

h_x = F.softmax(logit, dim=1).data.squeeze()
probs, idx = h_x.sort(0, True)
probs = probs.numpy()
idx = idx.numpy()

# output the prediction
for i in range(0, 5):
    print('{:.3f} -> {}'.format(probs[i], classes_ja[idx[i]]))

0.327 -> ダルメシアン
0.064 -> ラブラドル・レトリーバー犬
0.042 -> 蟻
0.033 -> ヒップ
0.032 -> イングリッシュフォックスハウンド


In [ ]:
# generate class activation mapping for the top1 prediction
candidate = 1
CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[candidate]])

# render the CAM and output
#print('output CAM.jpg for the top1 prediction: %s'%classes[idx[0]])
print('output CAM.jpg for the top1 prediction: %s'%classes_ja[idx[candidate]])

img = cv2.imread('test.jpg')
height, width, _ = img.shape
heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)
result = heatmap * 0.3 + img * 0.5
cv2.imwrite('CAM.jpg', result)
IPython.display.Image('CAM.jpg')

In [20]:
# まとめて関数化
def CAM5(img_url, n=5):
    response = requests.get(img_url)
    img_pil = Image.open(io.BytesIO(response.content))
    img_pil.save('test.jpg')
    
    img_tensor = preprocess(img_pil)
    img_variable = Variable(img_tensor.unsqueeze(0))
    logit = net(img_variable)

    h_x = F.softmax(logit, dim=1).data.squeeze()
    probs, idx = h_x.sort(0, True)
    probs = probs.numpy()
    idx = idx.numpy()

    for cand in range(n):
        # generate class activation mapping for the top1 prediction
        CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[cand]])

        # render the CAM and output
        print('CAM の出力 {0}: {1}'. format(candidate,  classes[idx[cand]]))

        img = cv2.imread('test.jpg')
        height, width, _ = img.shape
        heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)
        result = heatmap * 0.3 + img * 0.5
        cv2.imwrite('CAM.jpg', result)
        #IPython.display.Image('CAM.jpg')
        plt.axis(False); plt.imshow(pil_img.open('CAM.jpg'))
        plt.show()


In [ ]:
url_img = 'https://raw.githubusercontent.com/komazawa-deep-learning/komazawa-deep-learning.github.io/master/assets/2012AlexNetResult.jpg'
CAM5(url_img)